# 1 . Importing all required libraries

In [1]:
import cv2
import numpy as np
from gtts import gTTS
import os

# 2. Importing Datasets

In [2]:
net = cv2.dnn.readNet('./yolov3.weights','./yolov3.cfg')
classes = []

with open('./coco.names' , 'r') as f:
    classes = f.read().splitlines()


# 3 . Capturing object 

In [3]:
cap  =  cv2.VideoCapture(0)
ret , photo  =  cap.read()
cv2.imwrite("my.jpg" , photo)
cap.release()
cv2.waitKey()
cv2.destroyAllWindows()

# 4 . Reading image and shapping image

In [4]:
img = cv2.imread('my.jpg')
height , width , _ = img.shape

In [5]:
blob = cv2.dnn.blobFromImage(img, 1/255,(416,416),(0,0,0), swapRB=True , crop=False)
net.setInput(blob)
output_layers_names = net.getUnconnectedOutLayersNames()
layersOutputs = net.forward(output_layers_names)

In [6]:
boxes = []
confidences = []
class_ids = []

In [8]:
for output in layersOutputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            centre_x = int(detection[0]*width)
            centre_y = int(detection[1]*height)
            w = int(detection[2]*width)
            h = int(detection[3]*height)
            
            
            x = int(centre_x - w/2)
            y = int(centre_y - h/2)
            
            boxes.append([x,y,w,h])
            confidences.append((float(confidence)))
            class_ids.append(class_id)

In [9]:
texts = []
indexes = cv2.dnn.NMSBoxes(boxes,confidences, 0.5, 0.4)
font = cv2.FONT_HERSHEY_PLAIN
colors =  np.random.uniform(0,255, size=(len(boxes) , 3))


In [10]:
for i in indexes.flatten():
    x,y,w,h = boxes[i]
    label = str(classes[class_ids[i]])
    confidence = str(round(confidences[i] ,2))
    color = colors[i]
    cv2.rectangle(img,(x,y) , (x+w , y+h), color ,2)
    cv2.putText(img , label+ " " + confidence, (x,y+20) , font , 2 , (255,255,255) , 2)
    texts.append(label)


In [11]:
if texts:
                description = ', '.join(texts)
                tts = gTTS("This is a " + description, lang='en')
                tts.save('tts.mp3')
                os.system("start tts.mp3")

In [12]:
cv2.imshow('Image',img)
cv2.waitKey(0)
#cap.release()
cv2.destroyAllWindows()